In [1]:
import torch
from torch import nn
from d2l import torch as d2l

train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size = 256, resize=(224, 224))
X_example, y_example = next(iter(train_iter))
print(X_example.shape, y_example.shape)

torch.Size([256, 1, 224, 224]) torch.Size([256])


In [3]:
def vgg_block(num_convs, in_channels, out_channels):
    layers  = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, 
                                kernel_size = 3, padding = 1))
        layers.append(nn.ReLU())
        in_channels = out_channels
        
    layers.append(nn.MaxPool2d(kernel_size = 2, stride = 1))
    return nn.Sequential(*layers)

In [8]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
        
    return nn.Sequential(*conv_blks, nn.Flatten(), 
                         nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(),
                         nn.Dropout(0.5), nn.Linear(4096, 4096), nn.ReLU(),
                         nn.Dropout(0.5), nn.Linear(4096, 10))

In [9]:
net = vgg(conv_arch)
# 获取参数大小和参数数量
total_params = 0
for name, param in net.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Count: {param.nelement()} | {param.data.dtype}")
    total_params += param.nelement()

print(f"Total parameters count: {total_params}")

Layer: 0.0.weight | Size: torch.Size([64, 1, 3, 3]) | Count: 576 | torch.float32
Layer: 0.0.bias | Size: torch.Size([64]) | Count: 64 | torch.float32
Layer: 1.0.weight | Size: torch.Size([128, 64, 3, 3]) | Count: 73728 | torch.float32
Layer: 1.0.bias | Size: torch.Size([128]) | Count: 128 | torch.float32
Layer: 2.0.weight | Size: torch.Size([256, 128, 3, 3]) | Count: 294912 | torch.float32
Layer: 2.0.bias | Size: torch.Size([256]) | Count: 256 | torch.float32
Layer: 2.2.weight | Size: torch.Size([256, 256, 3, 3]) | Count: 589824 | torch.float32
Layer: 2.2.bias | Size: torch.Size([256]) | Count: 256 | torch.float32
Layer: 3.0.weight | Size: torch.Size([512, 256, 3, 3]) | Count: 1179648 | torch.float32
Layer: 3.0.bias | Size: torch.Size([512]) | Count: 512 | torch.float32
Layer: 3.2.weight | Size: torch.Size([512, 512, 3, 3]) | Count: 2359296 | torch.float32
Layer: 3.2.bias | Size: torch.Size([512]) | Count: 512 | torch.float32
Layer: 4.0.weight | Size: torch.Size([512, 512, 3, 3]) | Cou

In [ ]:
for model in net:
    X_example = model(X_example)
    print(model.__class__.__name__, X_example.shape)

In [12]:
help(d2l.train_ch6)

Help on function train_ch6 in module d2l.torch:

train_ch6(net, train_iter, test_iter, num_epochs, lr, device)
    Train a model with a GPU (defined in Chapter 6).
    
    Defined in :numref:`sec_lenet`



In [13]:
d2l.train_ch6(net, train_iter, test_iter, 10, 0.1, "cuda")

training on cuda


RuntimeError: CUDA out of memory. Tried to allocate 100.00 MiB (GPU 0; 1.83 GiB total capacity; 14.21 MiB already allocated; 89.75 MiB free; 22.00 MiB reserved in total by PyTorch)